In [113]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow.keras import models, layers
from keras.models import Sequential 
from keras.layers import Dense, Dropout, Flatten 
from keras.layers import Conv2D, MaxPooling2D 
import cv2 as cv 
import matplotlib.pyplot as plt
import glob

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [114]:
cats_og=[]
dogs_og=[]
folder_cat = glob.glob('../input/microsoft-catsvsdogs-dataset/PetImages/Cat/')
for f in glob.glob(folder_cat[0]+'*.jpg'):
    cats_og.append(f)
    
folder_dog = glob.glob('../input/microsoft-catsvsdogs-dataset/PetImages/Dog')
for f in glob.glob(folder_dog[0]+'/*.jpg'):
    dogs_og.append(f)

In [115]:
data=[]
for i in range(len(cats_og)):
    try:
        img=cv.imread(cats_og[i])
        img=cv.resize(img, (64,64), interpolation=cv.INTER_AREA)
        img=np.array(img)
        print(i, np.shape(img))
        data.append([img,np.array([1,0])])
    except:
        print("Caught exception")
    
for i in range(len(dogs_og)):
    try:
        img=cv.imread(dogs_og[i])
        img=cv.resize(img, (64,64), interpolation=cv.INTER_AREA)
        img=np.array(img)
        print(i, np.shape(img))
        data.append([img,np.array([0,1])])
    except:
        print("Caught exception")
        
    

In [116]:
np.random.shuffle(data)
dataset=np.array(data, dtype=object)
dataset=dataset.T
X_train=np.stack(dataset[0])[:20000,:,:,:]
Y_train=np.stack(dataset[1])[:20000]
X_test=np.stack(dataset[0])[20000:,:,:,:]
Y_test=np.stack(dataset[1])[20000:]

In [117]:
model = Sequential()
model.add(Conv2D(64,(3,3), activation='relu', input_shape=(64,64,3)))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(128,(3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(layers.Flatten())
model.add(Dense(64, activation='softmax'))
model.add(Dense(2))

In [118]:
X_train=X_train/255
X_test=X_test/255

In [119]:
optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer,
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [120]:
model.fit(
   X_train, Y_train, 
   epochs = 10, 
   validation_data = (X_test, Y_test)
)